This is an inference notebook.
The notebook to train the model can be found [here](https://www.kaggle.com/palash97/gbr-fasterrcnn-pytorch-training/)

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import torch
import torchvision

from albumentations.pytorch.transforms import ToTensorV2
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN

## Load fasterrcnn model

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)
model = model.to(device)

In [ ]:
num_classes = 2  # 1 class (cots) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes).to(device)

## Load our trained weights 

In [ ]:
best_model = '../input/gbr-fasterrcnn-models/gbr_fasterrcnn_resnet50_fpn.pth.tar'
model.load_state_dict(torch.load(best_model, map_location=device))

## Make submission

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()
iter_test = env.iter_test()

In [ ]:
# define object detection threshold
threshold = 0.45

In [ ]:
for idx, (image, pred_df) in enumerate(iter_test):
    
    image = image.astype(np.float32) / 255.0
    image = ToTensorV2()(image=image)['image']
    images = image.unsqueeze(0)   # (1, 3, 720, 1280)
    
    model.eval()
    
    # Forward pass
    output = model(images.to(device))[0]
    
    # output is a dictionary
    boxes = output['boxes'].detach().cpu().numpy()
    scores = output['scores'].detach().cpu().numpy()
    
    boxes = boxes[scores >= threshold].astype(np.int32)
    scores = scores[scores >= threshold]

    # Convert xyxy to xywh
    boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
    
    predictions = []

    for i in zip(scores, boxes):
        confidence_score = i[0]
        x_min = i[1][0]
        y_min = i[1][1]
        width = i[1][2]
        height = i[1][3]
        predictions.append(f'{confidence_score:.2f} {x_min} {y_min} {width} {height}')

    pred_df['annotations'] = ' '.join(predictions)
    env.predict(pred_df)

In [ ]:
print('Done!')